In [ ]:
# import nessessory library
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/cancerChatbot.json').read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['question']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['context']))

        # adding classes to our class list
        if intent['context'] not in classes:
            classes.append(intent['context'])
        print(pattern)
        

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

Hi
How are you
Is anyone there?
Hello
Good day
Bye
See you later
Goodbye
Nice chatting to you, bye
Till next time
Thanks
Thank you
That's helpful
Awesome, thanks
Thanks for helping me
How you could help me?
What you can do?
What help you provide?
How you can be helpful?
What support is offered
who are you?
what is lungs cancer?
lungs cancer?
پھیپھڑوں کا کینسر کیا ہے؟
پھیپھڑوں کا کینسر کیسے پھیلتا ہے۔
lungs cancer kyia hy?
How does lungs cancer spread?
lungs cancer spread?
spread lungs cancer ?
پھیپھڑوں کا کینسر کیسے پھیلتا ہے۔
lungs cancer ksay phelta hy?
What are the symptoms of lungs cancer?
symptoms of lungs cancer?
پھیپھڑوں کے کینسر کی علامات کیا ہیں؟
lungs cancer symptoms?
lungs cacner ka symptoms kyia hn?
What are the Causes of lung cancer?
Causes of lung cancer?
پھیپھڑوں کے کینسر کی وجوہات کیا ہیں؟
lungs cancer kin wajha sa phelta hy?
lungs cancer causes
How do I know if it is lungs cancer or just the common cough?
lungs cancer cough or normal cough
normal cough or lungs cancer 

In [13]:
for intent in intents['intents']:
    for pattern in intent['question']:
      print(pattern)

Hi
How are you
Is anyone there?
Hello
Good day
Bye
See you later
Goodbye
Nice chatting to you, bye
Till next time
Thanks
Thank you
That's helpful
Awesome, thanks
Thanks for helping me
How you could help me?
What you can do?
What help you provide?
How you can be helpful?
What support is offered
who are you?
what is lungs cancer?
lungs cancer?
پھیپھڑوں کا کینسر کیا ہے؟
پھیپھڑوں کا کینسر کیسے پھیلتا ہے۔
lungs cancer kyia hy?
How does lungs cancer spread?
lungs cancer spread?
spread lungs cancer ?
پھیپھڑوں کا کینسر کیسے پھیلتا ہے۔
lungs cancer ksay phelta hy?
What are the symptoms of lungs cancer?
symptoms of lungs cancer?
پھیپھڑوں کے کینسر کی علامات کیا ہیں؟
lungs cancer symptoms?
lungs cacner ka symptoms kyia hn?
What are the Causes of lung cancer?
Causes of lung cancer?
پھیپھڑوں کے کینسر کی وجوہات کیا ہیں؟
lungs cancer kin wajha sa phelta hy?
lungs cancer causes
How do I know if it is lungs cancer or just the common cough?
lungs cancer cough or normal cough
normal cough or lungs cancer 

In [ ]:
print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


140 documents
32 classes [' higher risk for lung cancer   کن لوگوں کو پھیپھڑوں کے کینسر کا زیادہ خطرہ ہے kin logo ko lungs cancer ka zayida khatra hy', 'Are antibiotics effective in preventing or treating lungs cancer   antibiotic lungs cancer pa effect krti hy    کیا اینٹی بائیوٹکس پھیپھڑوں کے کینسر کی روک تھام یا علاج میں موثر ہیں؟', 'Breathing changes in lungs cancer khansi ki tabdeeli ka ksay pta chla ga  shortness of breath', 'Causes of lung cancer', 'Differences in survival rate between women and men men or women mn survival rate kitna hy    خواتین اور مردوں کے درمیان بقا کی شرح میں فرق؟', 'Early signs and symptoms of lung cancer  lungs cancer ki phali stage kyia hy?  first stage of lungs cancer symptoms  پھیپھڑوں کے کینسر کی ابتدائی علامات اور علامات', 'Effects of smoking for women vs. men  men or women mn smoking ka kyia nuksanat hn?  خواتین بمقابلہ مردوں پر سگریٹ نوشی کے کیا اثرات ہیں', 'End-stage last symptoms for lung cancer  lungs cancer ki akhri stage kyia hy  پھیپھڑوں کے 

In [ ]:
pickle.dump(words,open('cancer_care_words1.1.pkl','wb'))
pickle.dump(classes,open('cancer_care_classes1.1.pkl','wb'))

In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [ ]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
#fitting model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=16, verbose=1)

Epoch 1/200
9/9 [==============================] - 3s 4ms/step - loss: 3.5003 - accuracy: 0.0214 
Epoch 2/200
9/9 [==============================] - 0s 4ms/step - loss: 3.4691 - accuracy: 0.0571
Epoch 3/200
9/9 [==============================] - 0s 4ms/step - loss: 3.4442 - accuracy: 0.0643
Epoch 4/200
9/9 [==============================] - 0s 5ms/step - loss: 3.3761 - accuracy: 0.0429
Epoch 5/200
9/9 [==============================] - 0s 5ms/step - loss: 3.3710 - accuracy: 0.0643
Epoch 6/200
9/9 [==============================] - 0s 5ms/step - loss: 3.3309 - accuracy: 0.0786
Epoch 7/200
9/9 [==============================] - 0s 4ms/step - loss: 3.3271 - accuracy: 0.0857
Epoch 8/200
9/9 [==============================] - 0s 4ms/step - loss: 3.3026 - accuracy: 0.0857
Epoch 9/200
9/9 [==============================] - 0s 4ms/step - loss: 3.2126 - accuracy: 0.1857
Epoch 10/200
9/9 [==============================] - 0s 4ms/step - loss: 3.2013 - accuracy: 0.1929
Epoch 11/200
9/9 [==========

In [ ]:
# saving the model
model.save('cancer_care_model1.1.h5', hist)

print("model created")

model created


In [ ]:
from flask import Flask, jsonify
# from flask_cors import CORS, cross_origin
import random
import json
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


intents = json.loads(open('/content/intents4.json', encoding="utf8").read())
words = pickle.load(open('/content/final_words.pkl', 'rb'))
classes = pickle.load(open('/content/final_classes.pkl', 'rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['context'] == tag):
            result = random.choice(i['answer'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

def decrypt(msg):

    string = msg

    # converting back '+' character back into ' ' spaces
    # new_string is the normal message with spaces that was sent by the user
    new_string = string.replace("+", " ")

    return new_string

FileNotFoundError: ignored

In [ ]:
!pip install -U flask-cors


In [ ]:
from flask import Flask, jsonify
from flask_cors import CORS, cross_origin
import random
import json
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#from keras.models import load_model
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

model = load_model('/content/final_chatbot_model1.h5')

intents = json.loads(open('/content/intents4.json', encoding="utf8").read())
words = pickle.load(open('/content/final_words.pkl', 'rb'))
classes = pickle.load(open('/content/final_classes.pkl', 'rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['context'] == tag):
            result = random.choice(i['answer'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


''' Flask code '''


app = Flask(__name__)
CORS(app)


@app.route("/", methods=['GET', 'POST'])
def hello():
    return jsonify({"key": "home page value"})

# function to replace '+' character with ' ' spaces


def decrypt(msg):

    string = msg

    # converting back '+' character back into ' ' spaces
    # new_string is the normal message with spaces that was sent by the user
    new_string = string.replace("+", " ")

    return new_string

# here we will send a string from the client and the server will return another
# string with som modification
# creating a url dynamically


@app.route('/home/<name>')
def hello_name(name):

    # dec_msg is the real question asked by the user
    dec_msg = decrypt(name)

    # get the response from the ML model & dec_msg as the argument
    response = chatbot_response(dec_msg)

    # creating a json object
    json_obj = jsonify({"top": {"res": response}})

    return json_obj


if __name__ == '__main__':
    app.run(debug=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
